# Spring's constant measuring

## Importing libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import pandas as pd
import os
from functools import reduce

from modules import LinearRegression

## Load data

The following code creates a table (dict) with entries that have:
- as key, the name of the file from which the data is read
- as value, the DataFrame returned by pandas when reading that file

Getting data from this table involves using the file from which the data is read as key and then parsing the columns of the corresponding DataFrame.

e.g.:

```
masses_measurements = datasets['misure_masse']
used_objects = masses_measurements[masses_measurements.columns[0]]
masses = masses_measurements[masses_measurements.columns[1]]
```

In [ ]:
# collect filenames of exported data

data_dir = "./data"

csv_data = []
excel_data = []

for root, dirs, files in os.walk(data_dir):
    for file in files:
        current_file_path = os.path.join(root, file).replace(
            "\\", "/"
        )  # fix the unbearably frustrating flaws of the wanna-be OS... "Windows"
        if "xlsx" in file:
            excel_data.append(current_file_path)
        elif "csv" in file:
            csv_data.append(current_file_path)
    break  # stop at first recursion level: only ./data

print(f"CSV:\n{csv_data}\nEXCEL:\n{excel_data}")

In [ ]:
def read_csv(filename: str):
    data = pd.read_csv(filename, sep=";").replace(",", ".", regex=True)

    data.dropna(inplace=True)
    data.drop(
        index=data.index[0], axis=0, inplace=True
    )  # instrument error causes first value to be nonsensical

    for col in data.columns:
        data[col] = data[col].apply(pd.to_numeric)

    return data


def read_excel(filename: str):
    data = pd.read_excel(filename)
    return data


datasets = dict()

for file in csv_data + excel_data:
    key = os.path.basename(file).split(".")[0]
    if "csv" in file:
        datasets[key] = read_csv(file)
    elif "xlsx" in file:
        datasets[key] = read_excel(file)

print(datasets.keys())

for k in datasets.keys():
    print(f"\nkey: {k}")
    datasets[k].info()

## Analyze data

For the **static method**, the sonar's signal is hypothesized to have a very small amplitude, so the measure of the mass' displacement is the mean of the recorded positions.<BR><BR>
For the **dynamic method**, we isolate the peaks of the oscillating signal and compute the mean of the period between two adjacent peaks. From there, the resulting period is used to approximate the frequency of the oscillation.

In [ ]:
static_keys = list(filter(lambda k: "statico" in k, datasets.keys()))
dynamic_keys = list(filter(lambda k: "dinamico" in k, datasets.keys()))

print(f"{static_keys}\n{dynamic_keys}")

static_datasets = {k: datasets[k] for k in static_keys}
dynamic_datasets = {k: datasets[k] for k in dynamic_keys}

masses = datasets["misure_masse"]  # it's going to be used a lot, so...

### Static method

In [ ]:
# Analyze static method

averages = np.zeros(7)
for run in range(7):
    averages[run] += np.mean(data[f"Run{run+1}(mm)"])

averages = np.round_(averages, decimals=2)
print(averages)

# k = 9.81 * (masse[-1] - masse[0]) / (averages[-1] - averages[0])

### Dynamic method

In [ ]:
# Analyze dynamic method


def discrete_derivative(domain: list, curve: list):
    d = [0]
    for i in range(1, len(curve)):
        d.append((curve[i] - curve[i - 1]) / (domain[i] - domain[i - 1]))
    return d


# windowed peaks isolation
def windowed_peaks(signal: list):
    peaks = []
    for i in range(2, len(signal) - 2, 4):
        if (
            reduce(
                lambda x, y: x * y,
                discrete_derivative(list(range(i - 2, i + 3)), signal[i - 2 : i + 3]),
            )
            < 0
        ):
            peaks.append(max(signal[i - 2 : i + 3]))
    return peaks

In [ ]:
def calculate_mean_period(column_id: str):
    wf = data[column_id].array
    for v in wf:
        print(windowed_peaks(wf))


# calculate_mean_period(data[data.columns[0]])

# for col in data.columns:
# calculate_mean_period(col)